# Notebook for making Ungulate Threat to GDEs

Name:        Threats to GDEs due to ungulates

Purpose:     Determine whether GDEs are at risk of damage from ungulates (depends on GDE type and ungulate potential presence)
              
Refer to spreadsheet for notes on affected systems:  D:\GDE_Threats\OtherData\Grazing\grazing_impacts_by_GDE.xlsx

GDE methods report has descriptions of systems for each layer: https://www.conservationgateway.org/ConservationByGeography/NorthAmerica/UnitedStates/nevada/water/Documents/NV_iGDE_MethodsReport.pdf

Author:      sarah.byer

Created:     October 2021


## Set up environments in ArcGIS Pro

In [1]:
# Import ArcGIS modules and check out spatial analyst extension
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

'CheckedOut'

In [2]:
# Path to temporary geodatabase
#path =  r"folder_path\gdbname.gdb"
path =  r"K:\GIS3\Projects\GDE\Maps\GDE_Threats\GDE_Threats.gdb"

# Environment settings
env.workspace = path
env.overwriteOutput = True
env.outputCoordinateSystem = arcpy.SpatialReference(26911) # Spatial reference NAD 1983 UTM Zone 11N. The code is '26911'

In [5]:
# Custom function that converts tables to pandas dataframe

def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)]
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields, where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe

## Read in ungulate data

In [ ]:

graze = arcpy.CopyFeatures_management(r'D:\GDE_Threats\OtherData\Grazing\BLM_National_Grazing_Allotments\gra.gdb\gra_allot_poly', 'graze')
hma = arcpy.CopyFeatures_management(r'D:\GDE_Threats\OtherData\BLM_National_Wild_Horse_and_Burro\blm_natl_whb_geocortex.gdb\whb_hma_pop_poly', 'hma')
elk = arcpy.CopyFeatures_management(r'D:\GDE_Threats\OtherData\Occupied_Elk_Distribution.shp', 'elk')
